In [1]:
import pandas as pd
import xgboost as xgb
import sys,random
import pickle
import os
import numpy as np

In [2]:
if not os.path.exists('train_33465_preds'):
    os.mkdir('train_33465_preds')

In [4]:
train_x_d_cols = pd.read_csv('./rank_d_feature_score.csv')
train_x_d_cols = list(train_x_d_cols.iloc[10:810].feature)

In [1]:
%%time

#load data
unlabel_date = pd.read_csv('../../preprocess_data/unlabel_x_date.csv')
unlabel_null = pd.read_csv('../../preprocess_data/unlabel_x_filter_null.csv').drop(columns=['id'])
unlabel_int = pd.read_csv('../../preprocess_data/unlabel_x_int.csv').drop(columns=['id','tag'])
unlabel_d = pd.read_csv('../../preprocess_data/unlabel_x_float_rank_d.csv',usecols=train_x_d_cols)
unlabel_nd = pd.read_csv('../../preprocess_data/unlabel_x_float_rank_nd.csv').drop(columns=['id'])
unlabel = pd.concat([unlabel_date,unlabel_null,unlabel_int,unlabel_d,unlabel_nd],axis=1,ignore_index=True,copy=False)
# pd.concat后特征名没有了，变成数字了
test_id = unlabel.iloc[:,0].values
test = unlabel.drop(0,axis=1)
dtest = xgb.DMatrix(test.values)

In [26]:
def pipeline(iteration,dtest,test_id):
    bst = xgb.Booster(model_file = './model/xgb{0}.model'.format(0))

    #predict test set
    test_y = bst.predict(dtest)
    test_result = pd.DataFrame(test_id,columns=["id"])
    test_result['score'] = test_y
    test_result.to_csv("./train_33465_preds_preds/xgb{0}.csv".format(iteration),index=None,encoding='utf-8')
    

In [27]:
%%time
for i in range(36):
    pipeline(i,dtest,test_id)

CPU times: user 7min 17s, sys: 21.3 s, total: 7min 38s
Wall time: 22.3 s


In [ ]:
files = os.listdir('train_33465_preds')
pred = pd.read_csv('./train_33465_preds/%s' %files[0])  
id = pred.id
score = pred.score
for file in files[1:]:
    score += pd.read_csv('./train_33465_preds/%s' %files[0]).score
score /= len(files)
avg_pred = pd.DataFrame(id,columns=['id'])
avg_pred['score'] = score

avg_pred.to_csv('./train_33465_avg_pred.csv', index=False)